<a href="https://colab.research.google.com/github/hdm23061993/stock-predictor/blob/main/stock-predictor-lstm-with-cuda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install alpaca-trade-api==1.4.0
!pip install torch==1.10.0+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html
!pip install torchvision==0.11.1+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html
!pip install pandas==1.3.5
!pip install numpy==1.19.5


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/cu113/torch_stable.html
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/cu113/torch_stable.html
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [14]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import alpaca_trade_api as tradeapi

# check if CUDA is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# define the LSTM model
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)

        out, _ = self.lstm(x, (h0, c0))
        out = out[:, -1, :]
        out = self.fc(out)
        return out

# set hyperparameters
input_size = 1
hidden_size = 64
num_layers = 2
output_size = 1
learning_rate = 0.001
num_epochs = 100

# connect to Alpaca API
API_KEY = 'PKC2GMYC3WMJU15DANOG'
SECRET_KEY = 'WAVhVD9y9yZUIYksrcb8PFofUhJMBBp7W3UcTBFd'
BASE_URL = 'https://paper-api.alpaca.markets'
api = tradeapi.REST(API_KEY, SECRET_KEY, api_version='v2')

# load the stock data
symbol = 'NVDA'

#barset = api.get_barset(symbol, 'day', limit=1000)
# nvda_bars = barset[symbol]

interval = '1D'
start_date = pd.Timestamp('2020-01-01', tz='America/New_York').isoformat()
end_date = pd.Timestamp('2022-03-31', tz='America/New_York').isoformat()
nvda_bars = api.get_barset(symbol, interval, start=start_date, end=end_date).df[symbol]

nvda_df = nvda_bars.df

# prepare the data
data = nvda_df['close'].values.reshape(-1, 1)
train_data_len = int(np.ceil(0.8 * len(data)))
train_data = data[0:train_data_len, :]

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)

train_data = scaled_data[0:train_data_len, :]
x_train = []
y_train = []

for i in range(60, len(train_data)):
    x_train.append(train_data[i-60:i, 0])
    y_train.append(train_data[i, 0])

x_train, y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

# create the LSTM model
model = LSTM(input_size, hidden_size, num_layers, output_size).to(device)

# set the loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# train the model
for epoch in range(num_epochs):
    inputs = torch.from_numpy(x_train).type(torch.Tensor).to(device)
    labels = torch.from_numpy(y_train).type(torch.Tensor).to(device)

    optimizer.zero_grad()
    outputs = model(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# make predictions on the test data
test_data = scaled_data[train_data_len-60:, :]
x_test = []
y_test = data[train_data_len:, :]

for i in range(60, len(test_data)):
    x_test.append


HTTPError: ignored